In [1]:
!pip install sentencepiece datasets

     |████████████████████████████████| 325 kB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 20.2 MB 8.1 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 18.0 MB/s eta 0:00:01
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)


In [19]:
sj_ngraphs = "ch sch sh si sj sk skj ssi ssj stj ti".split(" ")
j_digraphs = "dj gj hj lj".split(" ")
tj_digraphs = "kj tj".split(" ")
retroflex_digraphs = "rd rl rn rs rt".split(" ")
assimilations = "fv dt".split(" ")
diphthongs = "au eu".split(" ")
other = "ng gn ck".split(" ")
doublings = [f"{a}{a}" for a in "bcdfglmnpqrstvz"]

In [20]:
ngraphs = sj_ngraphs + j_digraphs + tj_digraphs + retroflex_digraphs + assimilations + diphthongs + doublings + other

In [21]:
import sentencepiece as spm

In [22]:
from datasets import load_dataset

dataset = load_dataset("common_voice", "sv-SE")

Reusing dataset common_voice (/Users/joregan/.cache/huggingface/datasets/common_voice/sv-SE/6.1.0/d3d5467c15716a2699f2ea3710fdc8bed7c20ae8ed66c248185735a0695dcc3b)
100%|██████████| 6/6 [00:00<00:00, 127.24it/s]


In [23]:
sentences = set()
_ALPHA = "abcdefghijklmnopqrstuvwxyzåäöéABCDEFGHIJKLMNOPQRSTUVWXYZÅÄÖ"
skipped = set()
for sentence in dataset["train"]["sentence"]:
    chars = []
    i = 0
    end = len(sentence)
    while i < end:
        if sentence[i] in ".,!?\"":
            skipped.add(sentence[i])
        elif sentence[i] == ":" or sentence[i] == "-":
            if (i + 1) < end and sentence[i+1] in _ALPHA:
                chars.append(sentence[i])
            else:
                skipped.add(sentence[i])
        else:
            chars.append(sentence[i])
        i += 1
    jnd = "".join(chars)
    sentences.add(jnd.lower())

In [25]:
with open("cv-text.txt", "w") as f:
    for sent in sentences:
        f.write(sent + "\n")

In [26]:
spm.SentencePieceTrainer.train(input="cv-text.txt", model_prefix="swedish1k", vocab_size=1000)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: cv-text.txt
  input_format: 
  model_prefix: swedish1k
  model_type: UNIGRAM
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
normalizer_spec {
  name: nmt_nfkc
  add_dummy_prefix: 1
  remove_extra_whitespaces: 1
  escape_w

In [27]:
spm.SentencePieceTrainer.train(input="cv-text.txt", model_prefix="swedish1k_aug", vocab_size=1000, user_defined_symbols=ngraphs)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: cv-text.txt
  input_format: 
  model_prefix: swedish1k_aug
  model_type: UNIGRAM
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: ch
  user_defined_symbols: sch
  user_defined_symbols: sh
  user_defined_symbols: si
  user_defined_symbols: sj
  user_defined_symbols: sk
  user_defined_symbols: skj
  user_defined_symbols: ssi
  user_defined_symbols: ssj
  user_defined_symbols: stj
  user_defined_symbols: ti
  user_defined_symbols: dj
  user_defined_symbols: gj
  user_defined_symbols: hj
 

In [31]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor(model_file="swedish1k.model")
sp2 = spm.SentencePieceProcessor(model_file="swedish1k_aug.model")
sp.encode("armén skickade hem honom eftersom han fungerade dåligt socialt", out_type=str)


['▁arm',
 'é',
 'n',
 '▁skicka',
 'de',
 '▁hem',
 '▁honom',
 '▁efter',
 's',
 'om',
 '▁han',
 '▁fungera',
 'de',
 '▁dålig',
 't',
 '▁social',
 't']

In [30]:
sp2.encode("armén skickade hem honom eftersom han fungerade dåligt socialt", out_type=str)


['▁arm',
 'é',
 'n',
 '▁',
 'sk',
 'i',
 'ck',
 'ade',
 '▁hem',
 '▁honom',
 '▁efter',
 'som',
 '▁han',
 '▁fu',
 'ng',
 'erade',
 '▁dåligt',
 '▁social',
 't']

In [32]:
spm.SentencePieceTrainer.train(input="cv-text.txt", model_prefix="swedish1kbpe", vocab_size=1000, model_type="bpe")

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: cv-text.txt
  input_format: 
  model_prefix: swedish1kbpe
  model_type: BPE
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
normalizer_spec {
  name: nmt_nfkc
  add_dummy_prefix: 1
  remove_extra_whitespaces: 1
  escape_wh

In [33]:
spm.SentencePieceTrainer.train(input="cv-text.txt", model_prefix="swedish1kbpe_aug", vocab_size=1000, model_type="bpe", user_defined_symbols=ngraphs)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: cv-text.txt
  input_format: 
  model_prefix: swedish1kbpe_aug
  model_type: BPE
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: ch
  user_defined_symbols: sch
  user_defined_symbols: sh
  user_defined_symbols: si
  user_defined_symbols: sj
  user_defined_symbols: sk
  user_defined_symbols: skj
  user_defined_symbols: ssi
  user_defined_symbols: ssj
  user_defined_symbols: stj
  user_defined_symbols: ti
  user_defined_symbols: dj
  user_defined_symbols: gj
  user_defined_symbols: hj
  

In [34]:
sp3 = spm.SentencePieceProcessor(model_file="swedish1kbpe_aug.model")
sp3.encode("armén skickade hem honom eftersom han fungerade dåligt socialt", out_type=str)


['▁ar',
 'm',
 'é',
 'n',
 '▁',
 'sk',
 'i',
 'ck',
 'ade',
 '▁hem',
 '▁honom',
 '▁efte',
 'rs',
 'om',
 '▁han',
 '▁fu',
 'ng',
 'erade',
 '▁då',
 'ligt',
 '▁so',
 'ci',
 'alt']

In [38]:
!pip install tokenizers protobuf

In [36]:
from tokenizers import SentencePieceUnigramTokenizer

In [39]:
tok1 = SentencePieceUnigramTokenizer.from_spm("swedish1k.model")
tok2 = SentencePieceUnigramTokenizer.from_spm("swedish1k_aug.model")
tok3 = SentencePieceUnigramTokenizer.from_spm("swedish1kbpe.model")
tok4 = SentencePieceUnigramTokenizer.from_spm("swedish1kbpe_aug.model")



Exception: You're trying to run a `Unigram` model but you're file was trained with a different algorithm

In [43]:
tok1.save_model("tok1")
tok2.save_model("tok2")

['tok2/unigram.json']